## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-15-01-02-35 +0000,nypost,Jasmine Crockett claims GOP would rather ‘bury...,https://nypost.com/2025/07/14/us-news/jasmine-...
1,2025-07-15-00-59-06 +0000,nyt,"Behind Trump’s Tough Talk on Russia and Putin,...",https://www.nytimes.com/2025/07/14/us/politics...
2,2025-07-15-00-53-28 +0000,nypost,NATO chief warns Russia’s trading partners aft...,https://nypost.com/2025/07/14/us-news/nato-chi...
3,2025-07-15-00-04-10 +0000,nypost,Teenage cleaner fatally sucked into meat grind...,https://nypost.com/2025/07/14/us-news/teenage-...
4,2025-07-14-23-53-29 +0000,nypost,Long Island school board accused of ‘blatant c...,https://nypost.com/2025/07/14/us-news/li-schoo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2316/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
13,trump,17
16,russia,8
191,ukraine,8
271,new,6
212,he,6


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
26,2025-07-14-21-34-30 +0000,bbc,Trump threatens Russia with tariffs while unve...,https://www.bbc.com/news/articles/czdv20v9lp1o,45
2,2025-07-15-00-53-28 +0000,nypost,NATO chief warns Russia’s trading partners aft...,https://nypost.com/2025/07/14/us-news/nato-chi...,44
41,2025-07-14-19-33-55 +0000,nypost,First lady Melania Trump helped convince presi...,https://nypost.com/2025/07/14/us-news/first-la...,43
35,2025-07-14-20-35-11 +0000,nyt,Trump Threatens Russia With Sanctions and Vows...,https://www.nytimes.com/2025/07/14/us/politics...,40
67,2025-07-14-11-20-22 +0000,nyt,Trump Is Expected to Announce New Weapons Pipe...,https://www.nytimes.com/2025/07/14/world/europ...,39


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
26,45,2025-07-14-21-34-30 +0000,bbc,Trump threatens Russia with tariffs while unve...,https://www.bbc.com/news/articles/czdv20v9lp1o
73,24,2025-07-14-05-26-09 +0000,nypost,Biden admits he didn’t approve each individual...,https://nypost.com/2025/07/14/us-news/biden-de...
24,23,2025-07-14-21-43-35 +0000,nypost,Texas flood volunteers ordered to evacuate as ...,https://nypost.com/2025/07/14/us-news/texas-fl...
15,22,2025-07-14-22-29-19 +0000,nypost,Founder of Cali pot farm where ICE rescued doz...,https://nypost.com/2025/07/14/us-news/founder-...
0,18,2025-07-15-01-02-35 +0000,nypost,Jasmine Crockett claims GOP would rather ‘bury...,https://nypost.com/2025/07/14/us-news/jasmine-...
28,18,2025-07-14-21-27-46 +0000,nypost,DOJ defends bringing case against Ghislaine Ma...,https://nypost.com/2025/07/14/us-news/doj-defe...
29,18,2025-07-14-21-22-29 +0000,nypost,New gov’t student-loan restrictions explained:...,https://nypost.com/2025/07/14/us-news/new-stud...
43,17,2025-07-14-19-28-42 +0000,nypost,Andrew Cuomo confirms independent run for NYC ...,https://nypost.com/2025/07/14/us-news/andrew-c...
16,17,2025-07-14-22-22-23 +0000,nypost,Blakeman renews ex-Rep. Peter King’s $8K a mon...,https://nypost.com/2025/07/14/us-news/blakeman...
2,15,2025-07-15-00-53-28 +0000,nypost,NATO chief warns Russia’s trading partners aft...,https://nypost.com/2025/07/14/us-news/nato-chi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
